In [3]:
import time
import pandas as pd
from datetime import datetime
import firebase_admin
from firebase_admin import credentials, firestore

In [4]:
cred = credentials.Certificate('../../src/config/smart-agri-bbd45-firebase-adminsdk-jfpri-5e03cc48f1.json')

app = firebase_admin.initialize_app(cred)
db = firestore.client()

In [5]:
bulk_writer = db.bulk_writer()
batch = db.batch()
collection = db.collection('emission_factors')


In [6]:
def create_data_item(row):
    return {
        "created_time": time.time(),
        "name": row.get("name", ""),
        "identity_title": row.get("identity_title", ""),
        "description": row.get("description", ""),
        "co2": float(row.get("co2", 0)),
        "co2_unit": row.get("co2_unit", "kg/ha"),
        "ch4": float(row.get("ch4", 0)),
        "ch4_unit": row.get("ch4_unit", "kg/ha"),
        "n2o": float(row.get("n2o", 0)),
        "n2o_unit": row.get("n2o_unit", "kg/ha"),
        "source": row.get("source", None),
        "link": row.get("link", None),
        "type": row.get("type", None),
        "scope": int(row.get("scope", 1)),
    }

In [7]:
def write_data_to_firestore(df):
    bulk_writer = db.bulk_writer()

    for index, row in df.iterrows():
        print(f"index: {index}")
        time.sleep(0.5)
        data_item = create_data_item(row)
        doc_ref = db.collection("emission_factors").document()
        print(f"data: {data_item}")
        print("________")
        bulk_writer.set(doc_ref, data_item)

    bulk_writer.flush()

In [8]:
df = pd.read_excel('../data/EF_Table.xlsx')

In [9]:
def process_name(name):
    words = name.split(" ")
    words = [word.lower() for word in words]
    title = "_".join(words)
    return title

In [10]:
df["identity_title"] = df["name"].apply(process_name)

In [11]:
df

,name,description,co2,co2_unit,ch4,ch4_unit,n2o,n2o_unit,source,link,type,scope,identity_title
0,Continuous Flooding,Continuous Flooding,0.000,kg,1.3000,kg,0.00000,kg,IPCC 2006 Guidelines,https://www.ipcc-nggip.iges.or.jp/public/2006gl/,Rice cultivation,1,continuous_flooding
1,Intermittent Flooding,Intermittent Flooding (Single aeration),0.000,kg,0.8000,kg,0.00000,kg,IPCC 2006 Guidelines,https://www.ipcc-nggip.iges.or.jp/public/2006gl/,Rice cultivation,1,intermittent_flooding
2,Intermittently Flooded Fields,Intermittently Flooded Fields (Multiple Aeration),0.000,kg,0.5000,kg,0.00000,kg,IPCC 2006 Guidelines,https://www.ipcc-nggip.iges.or.jp/public/2006gl/,Rice cultivation,1,intermittently_flooded_fields
3,Rainfed/Deep water,Rainfed/Deep water,0.000,kg,0.2000,kg,0.00000,kg,IPCC 2006 Guidelines,https://www.ipcc-nggip.iges.or.jp/public/2006gl/,Rice cultivation,1,rainfed/deep_water
4,Upland,Upland (Not flooded at all),0.000,kg,0.1000,kg,0.00000,kg,IPCC 2006 Guidelines,https://www.ipcc-nggip.iges.or.jp/public/2006gl/,Rice cultivation,1,upland
5,Straw incorporated shortly,Straw incorporated shortly (<30 days) before c...,0.000,kg,1.0000,kg,0.00000,kg,IPCC 2006 Guidelines,https://www.ipcc-nggip.iges.or.jp/public/2006gl/,Rice cultivation,1,straw_incorporated_shortly
6,Straw incorporated long,Straw incorporated long (>30 days) before cult...,0.000,kg,0.2900,kg,0.00000,kg,IPCC 2006 Guidelines,https://www.ipcc-nggip.iges.or.jp/public/2006gl/,Rice cultivation,1,straw_incorporated_long
7,Compost,Compost,0.000,kg,0.0500,kg,0.00000,kg,IPCC 2006 Guidelines,https://www.ipcc-nggip.iges.or.jp/public/2006gl/,Fertiliser / waste application,1,compost
8,Farm yard manure,Farm yard manure,0.000,kg,0.1400,kg,0.00000,kg,IPCC 2006 Guidelines,https://www.ipcc-nggip.iges.or.jp/public/2006gl/,Manure,1,farm_yard_manure
9,Green manure,Green manure,0.000,kg,0.5000,kg,0.00000,kg,IPCC 2006 Guidelines,https://www.ipcc-nggip.iges.or.jp/public/2006gl/,Manure,1,green_manure


In [12]:
df.shape

(23, 13)

In [13]:
# df = pd.read_excel('../data/EF_Table.xlsx')
start = time.time()
write_data_to_firestore(df)
end = time.time()
print(f'Execution time: {end - start} seconds')

index: 0
data: {'created_time': 1722042613.7115674, 'name': 'Continuous Flooding', 'identity_title': 'continuous_flooding', 'description': 'Continuous Flooding', 'co2': 0.0, 'co2_unit': 'kg', 'ch4': 1.3, 'ch4_unit': 'kg', 'n2o': 0.0, 'n2o_unit': 'kg', 'source': 'IPCC 2006 Guidelines', 'link': 'https://www.ipcc-nggip.iges.or.jp/public/2006gl/', 'type': 'Rice cultivation', 'scope': 1}
________
index: 1
data: {'created_time': 1722042614.211936, 'name': 'Intermittent Flooding', 'identity_title': 'intermittent_flooding', 'description': 'Intermittent Flooding (Single aeration)', 'co2': 0.0, 'co2_unit': 'kg', 'ch4': 0.8, 'ch4_unit': 'kg', 'n2o': 0.0, 'n2o_unit': 'kg', 'source': 'IPCC 2006 Guidelines', 'link': 'https://www.ipcc-nggip.iges.or.jp/public/2006gl/', 'type': 'Rice cultivation', 'scope': 1}
________
index: 2
data: {'created_time': 1722042614.7158923, 'name': 'Intermittently Flooded Fields', 'identity_title': 'intermittently_flooded_fields', 'description': 'Intermittently Flooded Fiel

In [24]:
df.to_excel("EF_Table_v2.xlsx", index=False)

In [ ]:
col_ref = db.collection("emission_factors")

In [ ]:
res = col_ref.stream()

In [ ]:
docs = [{"id": doc.id, **doc.to_dict()} for doc in res]

In [ ]:
docs